# Installation Etherpad-lite auf dem Pi

Grundlage dieser Anleitung ist https://github.com/ghoulmann/raspi-etherpad-lite

Zunächst die Paketedatenbank aktualisieren:

In [ ]:
sudo apt-get -qq update

Dann ggf. fehlende Pakete nachinstallieren.

In [ ]:
sudo apt-get install -y gzip git-core curl python libssl-dev pkg-config build-essential

Etherpad-lite benötigt node-js. Sollte installiert sein.

In [ ]:
node --version

Variablen mit dem Ort und dem Port. Danach wird etherpad-lite geklonet.

In [ ]:
target=/opt/etherpad-lite
port=8080
#git-clone to /opt/etherpad-lite
sudo git clone git://github.com/ether/etherpad-lite.git $target

Vorlage kopieren und danach den Port anpassen.

In [ ]:
#Create settings.json
sudo cp $target/settings.json.template $target/settings.json

In [ ]:
#configure etherpad
sudo sed -i "s|9001|$port|" $target/settings.json

Die Abhängigkeiten mit Skript nachinstallieren.

In [ ]:
#Make sure dependencies are up to date
sudo $target/bin/installDeps.sh

Ein Benuzter der etherpad-lite als Dienst laufen lässt.

ToDo: Das Anlegen eines Homeverzeichnises sollte für einen Systembenutzer nicht notwendig sein. Evtl auf /opt/etherpad-lite ändern.

In [ ]:
#useradd system user named etherpad-lite
sudo useradd -MrU etherpad-lite
sudo mkdir /home/etherpad-lite
sudo chowm etherpad-lite:

In [ ]:
#create log directory
sudo mkdir -p /var/log/etherpad-lite/

In [ ]:
#permissions
sudo chown -R etherpad-lite:etherpad-lite $target/
sudo chown -R etherpad-lite:etherpad-lite /var/log/etherpad-lite/

Erstellen eines Start-Stop (initV) Skriptes.

In [ ]:
#configure daemon
sudo su -c 'cat > /etc/init.d/etherpad-lite <<"EOF"
#!/bin/sh
### BEGIN INIT INFO
# Provides:          etherpad-lite
# Required-Start:    $local_fs $remote_fs $network $syslog
# Required-Stop:     $local_fs $remote_fs $network $syslog
# Default-Start:     2 3 4 5
# Default-Stop:      0 1 6
# Short-Description: starts etherpad lite
# Description:       starts etherpad lite using start-stop-daemon
### END INIT INFO
PATH="/usr/local/sbin:/usr/local/bin:/sbin:/bin:/usr/sbin:/usr/bin:/opt/node/bin"
LOGFILE="/var/log/etherpad-lite/etherpad-lite.log"
EPLITE_DIR="#target#"
EPLITE_BIN="bin/safeRun.sh"
USER="etherpad-lite"
GROUP="etherpad-lite"
DESC="Etherpad Lite"
NAME="etherpad-lite"
set -e
. /lib/lsb/init-functions
start() {
  echo "Starting $DESC... "
  
	start-stop-daemon --start --chuid "$USER:$GROUP" --background --make-pidfile --pidfile /var/run/$NAME.pid --exec $EPLITE_DIR/$EPLITE_BIN -- $LOGFILE || true
  echo "done"
}
#We need this function to ensure the whole process tree will be killed
killtree() {
    local _pid=$1
    local _sig=${2-TERM}
    for _child in $(ps -o pid --no-headers --ppid ${_pid}); do
        killtree ${_child} ${_sig}
    done
    kill -${_sig} ${_pid}
}
stop() {
  echo "Stopping $DESC... "
   while test -d /proc/$(cat /var/run/$NAME.pid); do
    killtree $(cat /var/run/$NAME.pid) 15
    sleep 0.5
  done
  rm /var/run/$NAME.pid
  echo "done"
}
status() {
  status_of_proc -p /var/run/$NAME.pid "" "etherpad-lite" && exit 0 || exit $?
}
case "$1" in
  start)
	  start
	  ;;
  stop)
    stop
	  ;;
  restart)
	  stop
	  start
	  ;;
  status)
	  status
	  ;;
  *)
	  echo "Usage: $NAME {start|stop|restart|status}" >&2
	  exit 1
	  ;;
esac
exit 0
EOF'

In [ ]:
#specify the etherpad install directory
sudo sed -i "s|#target#|$target|" /etc/init.d/etherpad-lite

In [ ]:
#Make daemon file executeable
sudo chmod +x /etc/init.d/etherpad-lite

In [ ]:
#Configure as a service
sudo update-rc.d etherpad-lite defaults

In [ ]:
sudo /etc/init.d/etherpad-lite start

In [ ]:
sudo /etc/init.d/etherpad-lite status

In [ ]:
nmap localhost

ToDo: Ändern auf mysql Datenbank.